-   <a href="#summary" id="toc-summary">Summary</a>
    -   <a href="#overview-and-explanation"
        id="toc-overview-and-explanation">Overview and Explanation</a>
-   <a href="#import-and-deterministic-setup"
    id="toc-import-and-deterministic-setup">Import and Deterministic
    Setup</a>
-   <a href="#data-preprocessing" id="toc-data-preprocessing">Data
    Preprocessing</a>
    -   <a href="#custom-dataset-class-to-load-covidnet-data"
        id="toc-custom-dataset-class-to-load-covidnet-data">Custom Dataset class
        to load CovidNet data</a>
    -   <a href="#spliting-dataset-into-train-val-test"
        id="toc-spliting-dataset-into-train-val-test">Spliting dataset into
        train, val, test</a>
    -   <a href="#applying-transforms-to-dataset"
        id="toc-applying-transforms-to-dataset">Applying transforms to
        dataset</a>
        -   <a href="#define-a-wrapper-dataset"
            id="toc-define-a-wrapper-dataset">Define a wrapper dataset</a>
        -   <a href="#defining-the-transforms"
            id="toc-defining-the-transforms">Defining the transforms</a>
        -   <a href="#creating-the-dataset-loader"
            id="toc-creating-the-dataset-loader">Creating the Dataset Loader</a>
-   <a href="#custom-model" id="toc-custom-model">Custom Model</a>
    -   <a href="#references" id="toc-references">References</a>
        -   <a href="#links" id="toc-links">Links</a>
    -   <a href="#first-try" id="toc-first-try">First try</a>
    -   <a href="#full-model" id="toc-full-model">Full Model</a>
-   <a href="#training-validation" id="toc-training-validation">Training
    &amp; Validation</a>
-   <a href="#test-evaluation" id="toc-test-evaluation">Test
    (Evaluation)</a>
-   <a href="#playground" id="toc-playground">Playground</a>
    -   <a href="#visualizing-models-using-pytorchviz"
        id="toc-visualizing-models-using-pytorchviz">Visualizing Models using
        PytorchViz</a>
    -   <a href="#investigate-image-in-dataset"
        id="toc-investigate-image-in-dataset">Investigate Image in Dataset</a>
    -   <a href="#investigate-metadata.csv"
        id="toc-investigate-metadata.csv">Investigate metadata.csv</a>
    -   <a href="#investigating-split-dataset"
        id="toc-investigating-split-dataset">Investigating split dataset</a>
    -   <a href="#visualizing-dataloader"
        id="toc-visualizing-dataloader">Visualizing Dataloader</a>
    -   <a href="#trying-avalance" id="toc-trying-avalance">Trying Avalance</a>
    -   <a href="#pretrained-model" id="toc-pretrained-model">Pretrained
        Model</a>
        -   <a href="#helper-functions" id="toc-helper-functions">Helper
            Functions</a>
        -   <a href="#define-function-to-initialize-deep-learning-models"
            id="toc-define-function-to-initialize-deep-learning-models">Define
            Function to Initialize Deep Learning Models</a>
        -   <a href="#define-function-to-train-models"
            id="toc-define-function-to-train-models">Define Function to Train
            Models</a>
        -   <a href="#define-functions-to-visualize-prediction"
            id="toc-define-functions-to-visualize-prediction">Define Functions to
            Visualize Prediction</a>
        -   <a href="#run-all-models---init-models-training"
            id="toc-run-all-models---init-models-training">Run all models - Init
            Models + Training</a>
    -   <a href="#out-of-memory-issue" id="out-of-memory-issue">Out of memory issue</a>

------------------------------------------------------------------------

# Summary

------------------------------------------------------------------------

> Expand to see summary and details

## Overview and Explanation

1.  This notebook reuses a lot of the [original transfer learning
    notebook](https://colab.research.google.com/drive/187Z04CNQBVV3jmCdA2sSbMTs9BB40qh_#scrollTo=QgZD08Q-YXXH)
    -   Here the focus is on building the new custom model using the
        CovidNet-CT database.
2.  The [`Setup Kaggle`](#scrollTo=wMQLloEgzPol) section:
    -   is not longer needed for notebook running in kaggle. Remained
        here for references only
    -   is where the dataset is being acquired.
    -   Explanation of various phases in the [CovidNet-CT ML
        code](https://github.com/haydengunraj/COVIDNet-CT/blob/02009821b9d063d01994cb70e61b8def0af275ab/run_covidnet_ct.py#L415):
        -   train phase is train phase
        -   test phase is validation phase
        -   infer phase is test phase
3.  The [`Data Preprocessing`](#scrollTo=JjsNA--kG9CV) section:
    -   refers to the way [CovidNet-CT preprocess its
        data](https://github.com/haydengunraj/COVIDNet-CT/blob/d1c6be5202a78d5f8802e40ff6b5b9d57189c797/dataset.py#L72)
    -   CovidNet-CT uses TensorFlow while this notebook adapts the code
        to use PyTorch
    -   Two highlights
        -   input shape is (512, 512, 3) instead of the (224, 224, 3)
            used by the imagenet model
        -   the image is cropped to the bounding box provided with the
            dataset before resize to 512x512
4.  The [`Training & Validation`](#scrollTo=YqGCBwYdasI_) section:
    -   refers to [how CovidNet-CT
        trains](https://github.com/haydengunraj/COVIDNet-CT/blob/02009821b9d063d01994cb70e61b8def0af275ab/run_covidnet_ct.py#L174)
    -   This part is almost identical to the original transfer learning
        model notebook.

------------------------------------------------------------------------

# Import and Deterministic Setup

------------------------------------------------------------------------

All modules will be imported here including modules used in the
[Playground](#playground) section

In [1]:
from __future__ import print_function, division
import os
%env CUDA_LAUNCH_BLOCKING=1
import random
import numpy as np
import torch

from os import path
import math

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from PIL import Image
from sklearn.model_selection import train_test_split #, StratifiedShuffleSplit

import torchvision
from torchvision import models, transforms #, datasets
import matplotlib.pyplot as plt

import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import time
import torch.nn.functional as F

import gc
import pandas as pd

# ensure reproducibility across different executions
# https://pytorch.org/docs/stable/notes/randomness.html
# https://www.kaggle.com/mlwhiz/third-place-model-for-toxic-comments-in-pytorch
# https://pytorch.org/docs/stable/generated/torch.use_deterministic_algorithms.html
SEED = 18
def seed_everything(seed=18):
    random.seed(seed)
    %env PYTHONHASHSEED=$seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
#torch.set_deterministic(True)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512
        
# https://www.kaggle.com/code/manabendrarout/vision-transformer-vit-pytorch-on-tpus-train/notebook
def is_tpu_avail():
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        TPU_DETECTED = True
    except:
        pass

    return TPU_DETECTED


def is_gpu_avail():
    GPU_DETECTED = False
    try:
        GPU_DETECTED = torch.cuda.is_available()
    except:
        pass

    return GPU_DETECTED

env: CUDA_LAUNCH_BLOCKING=1
env: CUBLAS_WORKSPACE_CONFIG=:4096:8
env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512


------------------------------------------------------------------------

# Data Preprocessing

------------------------------------------------------------------------

-   [how torch dataset is loaded](https://github.com/pytorch/vision/blob/main/torchvision/datasets/folder.py#L45)
-   [example custom model and custom dataset](https://github.com/ArnaudMallet/Plant_Patho/blob/master/Plant_Patho_4.ipynb)
    -   [pytorch thread](https://discuss.pytorch.org/t/how-to-load-data-from-a-csv/58315/10) that mentioned this example
-   [A well explained custom dataset](https://github.com/utkuozbulak/pytorch-custom-dataset-examples)

## Custom Dataset class to load CovidNet data

- Various references used: 
  - https://colab.research.google.com/drive/187Z04CNQBVV3jmCdA2sSbMTs9BB40qh\_#scrollTo=H9doKmx1TXK1 
  - https://drive.google.com/drive/folders/13PnDpSYUaVaKHjXjUK6bwWvJddDfbRad 
  - https://github.com/haydengunraj/COVIDNet-CT/blob/d1c6be5202a78d5f8802e40ff6b5b9d57189c797/dataset.py#L72 
  - https://github.com/haydengunraj/COVIDNet-CT/blob/master/docs/dataset.md 
  - https://www.kaggle.com/datasets/hgunraj/covidxct?select=metadata.csv 
  - https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets/50544887#50544887 
  - https://github.com/pytorch/vision/blob/main/torchvision/datasets/folder.py 
  - https://github.com/pytorch/vision/blob/d4a03fc02d0566ec97341046de58160370a35bd2/torchvision/datasets/vision.py#L10

In [3]:
class CovidNetDataset(Dataset):
    def __init__(self, img_dir, split_files, limit_size = 0, transform = None):
        # don't seem to need the csv file
        # self.df = pd.read_csv(csv_path)
        # _, self.class_to_idx  = self.find_classes(csv_path);

        self.img_dir = img_dir
        self.split_files = split_files
        
        self.size = 0
        self.limit_size = limit_size
        self.imgs, self.targets, self.bboxes = self.get_all_split_file_data()
        self.stradify_removal_based_on_limit()
        # self.imgs = [entry.name for entry in os.scandir(img_dir) if entry.is_file()]
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        # filename = self.df[index, "FILENAME"]
        # label = self.class_to_idx [self.df[index, "LABEL"]]
        # image = Image.open(os.path.join(self.img_dir, filename))

        label = self.targets[index]
        with open(self.imgs[index], "rb") as f:
            image = Image.open(f)
            image = image.crop(self.bboxes[index])
            image = image.copy()

        if self.transform is not None:
            image = self.transform(image)
    

        return image, label

    # from https://github.com/pytorch/vision/blob/main/torchvision/datasets/folder.py#L36
    def find_classes(self, csv_path=None):
        """Returns class name array and class_to_idx.
        See :class:`CovidNetDataset` for details.
        """
        # class_col = "finding"
        # classes = sorted(self.df[class_col].unique())
        # if not classes:
        #     raise FileNotFoundError(f"Couldn't find any class from '{class_col}' column in {csv_path}.")

        # hard code classes as the order are not alphabetic
        classes = ['Normal', 'Pneumonia', 'COVID-19']

        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def get_all_split_file_data(self):
        files, classes, bboxes = [], [], []
        for split_file in self.split_files:
            f, cls, bb = self.get_data_from_split_file(split_file)
            files.extend(f)
            classes.extend(cls)
            bboxes.extend(bb)
        return files, classes, bboxes

    # from https://github.com/haydengunraj/COVIDNet-CT/blob/d1c6be5202a78d5f8802e40ff6b5b9d57189c797/dataset.py#L108
    def get_data_from_split_file(self, split_file):
        """Gets image filenames, classes and bboxes"""
        files, classes, bboxes = [], [], []
        with open(split_file, 'r') as f:
            for line in f.readlines():
                fname, cls, xmin, ymin, xmax, ymax = line.strip('\n').split()
                files.append(path.join(self.img_dir, fname))
                classes.append(int(cls))
                bboxes.append([int(xmin), int(ymin), int(xmax), int(ymax)])
                self.size += 1
        return files, classes, bboxes
    
    '''Try to do stratified removal based on limit count if it is specified'''
    def stradify_removal_based_on_limit(self):
        _, class_to_idx = self.find_classes()
        MIN_SIZE = len(class_to_idx) * 10 # allow for some buffer to work with
        if self.limit_size <= 0 or self.limit_size <= MIN_SIZE or self.limit_size >= self.size:
            return
        
        total_remove_count = self.size - self.limit_size
        occurrence = {idx: self.targets.count(idx) for _, idx in class_to_idx.items()}
        target_remove_count = {idx: 0 for _, idx in class_to_idx.items()}
        for idx, count in occurrence.items():
            target_remove_count[idx] = math.floor(total_remove_count * count / self.size)
        
        print(occurrence)
        print(target_remove_count)
        
        for i in reversed(range(len(self.targets))):
            idx = self.targets[i]
            if target_remove_count[idx] > 0:
                del self.targets[i]
                del self.imgs[i]
                del self.bboxes[i]
                target_remove_count[idx] -= 1
                self.size -= 1

## Spliting dataset into train, val, test

-   [SO QA on spliting using sklearn](https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn)
    -   [Train test split example](https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test)
    -   [train test split example with indices](https://stackoverflow.com/questions/31521170/scikit-learn-train-test-split-with-indices)
-   [Pytorch stratified split example](https://discuss.pytorch.org/t/how-to-do-a-stratified-split/62290)
-   [sklearn StratifiedShuffleSplit doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)
-   [StratifiedShuffleSplit example](https://stackoverflow.com/questions/29438265/stratified-train-test-split-in-scikit-learn)
-   [another StratifiedShuffleSplit example](https://stackoverflow.com/questions/40829137/stratified-train-validation-test-split-in-scikit-learn)

In [4]:
# Specify all the filepath of the dataset
CURR_DIR = "/kaggle/working"
DATA_DIR = "/kaggle/input/covidxct"
dirs = [entry.name for entry in os.scandir(DATA_DIR) if entry.is_dir()]
assert len(dirs) == 1 # expects to only have 1 folder that contains all the images

DATASET_DIR = path.join(DATA_DIR, dirs[0])
METADATA_CSV = path.join(DATA_DIR, "metadata.csv")

DATASET_NAME = "COVIDx_CT-3A"
TRAIN_SPLIT_FILE = path.join(DATA_DIR, f"train_{DATASET_NAME}.txt")
VAL_SPLIT_FILE = path.join(DATA_DIR, f"val_{DATASET_NAME}.txt")
TEST_SPLIT_FILE = path.join(DATA_DIR, f"test_{DATASET_NAME}.txt")
SPLIT_FILES = [TRAIN_SPLIT_FILE, VAL_SPLIT_FILE, TEST_SPLIT_FILE]

MAX_COUNT = 10000
full_dataset = CovidNetDataset(DATASET_DIR, SPLIT_FILES, MAX_COUNT)
full_data_len = len(full_dataset)
print(f"Length of full dataset: {full_data_len}")

# # Defines ratios, w.r.t. whole dataset.
ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1
dummy_X = np.zeros(full_data_len)
indexes = np.arange(full_data_len)

# Produces test split. Uses train_test_split instead of StratifiedShuffleSplit to get x_remaining & y_remaining
# to be used in the next step. 
# Note that an additional indexes array is provided
x_remaining, _, y_remaining, _, temp_train_index, test_index = train_test_split(
    dummy_X, full_dataset.targets, indexes, test_size=ratio_test, stratify=full_dataset.targets, random_state=SEED)

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
_, _, _, _, train_index, val_index = train_test_split(
    x_remaining, y_remaining, temp_train_index, test_size=ratio_val_adjusted, stratify=y_remaining, random_state=SEED)

# dataset size
train_data_size = len(train_index)
valid_data_size = len(val_index)
test_data_size = len(test_index)

print(f"First 10 train_index: {train_index[:10]}")
print(f"length of train_index: {train_data_size}\n")
print(f"First 10 val_index: {val_index[:10]}")
print(f"length of val_index: {valid_data_size}\n")
print(f"First 10 test_index: {test_index[:10]}")
print(f"length of test_index: {test_data_size}\n")

{0: 71488, 1: 42943, 2: 310593}
{0: 69806, 1: 41932, 2: 303285}
Length of full dataset: 10001
First 10 train_index: [6373 6939 9338  105 7573 4773 9125 4210 4499 1754]
length of train_index: 7999

First 10 val_index: [6942 5767  536 6017 6502 1373 8798 3052 4665 9964]
length of val_index: 1001

First 10 test_index: [9022 9017 9868 1724  795 4754 4568 1985 9612 9534]
length of test_index: 1001



## Applying transforms to dataset

### Define a wrapper dataset

- This is to have the flexibility of applying different transforms to each of the splitted dataset 
- References 
    - [wrapper dataset source](https://stackoverflow.com/questions/57539567/augmenting-only-the-training-set-in-k-folds-cross-validation/57539790#57539790)
    - [pytorch dataset lazy loading idea](https://discuss.pytorch.org/t/split-dataset-into-training-and-validation-without-applying-training-transform/115429/3)
    - [individual transform using torchdata](https://stackoverflow.com/questions/61811946/train-valid-test-split-for-custom-dataset-using-pytorch-and-torchvision)

In [5]:
class WrapperDataset:
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.dataset)

### Defining the transforms

- References for mean and std of images 
    - [pytorch forum thread](https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/27?u=kharshit) 
    - [how the mean and std of imagenet transform being calculated](https://stackoverflow.com/questions/57532661/how-do-they-know-mean-and-std-the-input-value-of-transforms-normalize?noredirect=1&lq=1) 
    - [another similar SO question](https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2) 
    - [grayscale vs RGB images in ML training](https://towardsdatascience.com/transfer-learning-on-greyscale-images-how-to-fine-tune-pretrained-models-on-black-and-white-9a5150755c7a) 
    - Bounding box causing issue when batching as stacking don’t work with
    different size 
        - [easiest solution is to use tuple as the parameter](https://discuss.pytorch.org/t/runtimeerror-stack-expects-each-tensor-to-be-equal-size-but-got-3-224-224-at-entry-0-and-3-224-336-at-entry-3/87211/10) when calling `transform.resize()` 
        - [another solution is to override `collate_fn()`](https://discuss.pytorch.org/t/dataloader-gives-stack-expects-each-tensor-to-be-equal-size-due-to-different-image-has-different-objects-number/91941) when contructing `Dataloader`

In [6]:
covidnet_std_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((512, 512)), # this is important or else batching will have error due to bbox
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) # POTENTIAL_FINE_TUNE
])

covidnet_train_transform = transforms.Compose([
    transforms.RandomChoice(transforms=[
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=.3, hue=.3),
        transforms.RandomPerspective(distortion_scale=0.4),
        transforms.RandomAffine(degrees=(0, 0), translate=(0.05, 0.1), scale=(0.85, 0.95))])
    ])

image_transforms = {
    'train': transforms.Compose([
        covidnet_train_transform,
        covidnet_std_transform
    ]),
    'val': transforms.Compose([
        covidnet_std_transform
    ]),
    'test': transforms.Compose([
        covidnet_std_transform
    ]),
    'playground': transforms.Compose([
        covidnet_train_transform,
        covidnet_std_transform
    ])
}

### Creating the Dataset Loader

In [7]:
seed_everything(SEED)
BATCH_SIZE = 32

train_sampler = SubsetRandomSampler(train_index)
val_sampler = SubsetRandomSampler(val_index)
test_sampler = SubsetRandomSampler(test_index)

train_loader = DataLoader(WrapperDataset(full_dataset, image_transforms['train']), batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(WrapperDataset(full_dataset, image_transforms['val']), batch_size=BATCH_SIZE, sampler=val_sampler)
test_loader = DataLoader(WrapperDataset(full_dataset, image_transforms['test']), batch_size=BATCH_SIZE, sampler=test_sampler)

class_names, class_to_idx = full_dataset.find_classes()
print(class_names)
print(class_to_idx)

if is_tpu_avail():
    device = 'TPU'
elif is_gpu_avail():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')
print(f'train size:{train_data_size}; validation size:{valid_data_size}; test size:{test_data_size}')

env: PYTHONHASHSEED=18
['Normal', 'Pneumonia', 'COVID-19']
{'Normal': 0, 'Pneumonia': 1, 'COVID-19': 2}
Using device: cuda
train size:7999; validation size:1001; test size:1001


------------------------------------------------------------------------

# Custom Model

------------------------------------------------------------------------

- The design of this custom model is illustrated in a draw.io diagram 
    - [Onedrive shared file of all-cnn-diagram.drawio diagram](https://onedrive.live.com/?authkey=%21AL6NGGK0%5FDdNURY&cid=10930FD9F7DD82DD&id=10930FD9F7DD82DD%21226797&parId=10930FD9F7DD82DD%21226791&o=OneUp) 
    - [link to draw.io of the model](https://app.diagrams.net/#W10930fd9f7dd82dd%2F10930FD9F7DD82DD!226797)

## References

### Links

-   [10 CNN Architecture
    Illustrations](https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d#bca5)
    -   [Visualizing pytorch
        models](https://github.com/szagoruyko/pytorchviz)
-   Main model building references
    -   The [CT-3A github
        repo](https://github.com/haydengunraj/COVIDNet-CT/search?q=model)
        -   [tensorflow pretrained
            models](https://github.com/haydengunraj/COVIDNet-CT/blob/master/docs/models.md)
        -   How to [convert tensorflow checkpoints into pytorch
            format](https://github.com/lernapparat/lernapparat/blob/master/style_gan/pytorch_style_gan.ipynb)
            -   [pytorch
                thread](https://discuss.pytorch.org/t/loading-tensorflow-checkpoints-with-pytorch/151750)
        -   [pytorch
            thread](https://discuss.pytorch.org/t/combining-trained-models-in-pytorch/28383/44)
            about combining two existing models
    -   [Pytorch resnext50
        implementation](https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py#L792)
    -   [pytorch beginner tutorial on building
        model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)
-   Other model building references
    -   [Custom
        Resnet](https://github.com/Arijit-datascience/pytorch_cifar10/blob/main/model/custom_resnet.py)
    -   [Resnest convolution block
        code](https://github.com/CVHuber/Convolution/blob/main/ResNeSt%20Block.py)
    -   [A very clear implementation of InceptionV3](https://github.com/Moeo3/GoogLeNet-Inception-V3-pytorch/blob/master/googlenet_v3.py) that follows the naming of blocks in the diagram

## First try

In [ ]:
class CovidNetModel(nn.Module):
    def __init__(self):
        super().__init__() # same as super(CovidNetModel, self).__init__()
        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=5, stride=2, padding=5, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(3, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        return x

    def _make_block():
        pass

## Full Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class CovidNetModel(nn.Module):
    def __init__(self):
        super().__init__() # same as super(CovidNetModel, self).__init__()

------------------------------------------------------------------------

# Training & Validation

------------------------------------------------------------------------

------------------------------------------------------------------------

# Test (Evaluation)

------------------------------------------------------------------------

------------------------------------------------------------------------

# Playground

------------------------------------------------------------------------

> Testbed that is not compulsary for any part of this notebook

## Visualizing Models using PytorchViz

-   https://pytorch.org/docs/stable/nn.html
-   https://discuss.pytorch.org/t/combining-multiple-models-and-datasets/82623
-   [Mandrin explanation of pytorch resnet
    code](https://www.jianshu.com/p/90d61f53d15d)

In [ ]:
import torch
from torch import nn
%pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master
from torchviz import make_dot, make_dot_from_trace

In [ ]:
from torchvision import models
from torchsummary import summary

model = CovidNetModel()
print(model)
summary(model, (3, 224, 224))

render_model_pic_file = "resnext50_32x4d"
model = models.resnext50_32x4d()
summary(model, (3, 224, 224))

# x = torch.randn(1, 3, 224, 224).requires_grad_(True)
# y = model(x)
# dot = make_dot(y, params=dict(model.named_parameters()))

# dot.format = "png"
# dot.render(render_model_pic_file)
# files.download(f"{render_model_pic_file}.{dot.format}")

## Investigate Image in Dataset

In [ ]:
import subprocess

# Specify all the filepath of the dataset
DATA_DIR = "/kaggle/input/covidxct"
DATASET_DIR = path.join(DATA_DIR, "3A_images/")
DATASET_NAME = "COVIDx_CT-3A"
TEST_SPLIT_FILE = path.join(DATA_DIR, f"test_{DATASET_NAME}.txt")

# count number of images
# !ls -Uba1 /content/data/3A_images | grep -c png
ls = subprocess.Popen(("ls", "-Uba1", DATASET_DIR), stdout=subprocess.PIPE)
output = subprocess.check_output(("grep", "-c", "png"), stdin=ls.stdout)
print(f"number of images: {output.decode()}")

# list first 10 images
ls = subprocess.Popen(("ls", "-Uba1", DATASET_DIR), stdout=subprocess.PIPE)
output = subprocess.check_output(("head", "-10"), stdin=ls.stdout)
first_10_lines = output.decode()
# print(f"first 10 images in {DATASET_DIR}:\n{first_10_lines}")
img_list = first_10_lines.split('\n')
first_png = next(filter(lambda img: "png" in img, img_list), None)
print(f"first_png: {first_png}\n")

In [ ]:
# show the first test image, unbounded followed by bounded
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as torch_func_trans
from PIL import Image

# from https://github.com/haydengunraj/COVIDNet-CT/blob/d1c6be5202a78d5f8802e40ff6b5b9d57189c797/dataset.py#L108
def get_data_from_split_file(split_file):
    """Gets image filenames, classes and bboxes"""
    files, classes, bboxes = [], [], []
    with open(split_file, 'r') as f:
        for line in f.readlines():
            fname, cls, xmin, ymin, xmax, ymax = line.strip('\n').split()
            files.append(path.join(DATASET_DIR, fname))
            classes.append(int(cls))
            bboxes.append([int(xmin), int(ymin), int(xmax), int(ymax)])
    return files, classes, bboxes

def bbox_to_topLeftOrigin_size(xmin, ymin, xmax, ymax):
    top = ymax
    left = xmin
    height = ymax - ymin
    width = xmax - xmin
    return top, left, height, width

files, classes, bbox = get_data_from_split_file(TEST_SPLIT_FILE)
first_file_tuple = (files[0], classes[0], bbox[0])

print(f"first image: {first_file_tuple[0]}")
if 0:
    # this way of reading image is deprecated
    img = plt.imread(first_file_tuple[0])

    fig = plt.figure(figsize=(15,15))
    plt.title("unbounded")
    _ = plt.imshow(img)
    _ = plt.axis('off')

    torch_img = torch.from_numpy(img)
    print(f"torch_img size: {torch_img.size()}")
    pytorch_size = bbox_to_topLeftOrigin_size(*first_file_tuple[2])
    print(f"pytorch_size: {pytorch_size}")

    cropped_img = torch_func_trans.crop(torch_img, *pytorch_size)
    fig = plt.figure(figsize=(15,15))
    plt.title("bounded")
    _ = plt.imshow(cropped_img)
    _ = plt.axis('off')

# This is the recommended method for opening image
# https://pillow.readthedocs.io/en/stable/reference/Image.html#examples
with Image.open(first_file_tuple[0]) as im:
    print("\n")
    print("Unbounded image")
    IMG = im
    display(im)

    print("\n")
    print("Bounded image")
    display(im.crop(first_file_tuple[2]))

    # https://pillow.readthedocs.io/en/latest/reference/open_files.html#file-handling
    print("\n")
    print("Out of Scope Unbounded image")
    display(IMG)
    

## Investigate metadata.csv

In [ ]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)

# Specify all the filepath of the dataset
DATA_DIR = "/kaggle/input/covidxct"
dirs = [entry.name for entry in os.scandir(DATA_DIR) if entry.is_dir()]
assert len(dirs) == 1 # expects to only have 1 folder that contains all the images

DATASET_DIR = path.join(DATA_DIR, dirs[0])
METADATA_CSV = path.join(DATA_DIR, "metadata.csv")

meta_df = pd.read_csv(METADATA_CSV)
print("First 5 rows in metadata.csv0")
print(meta_df.head(5))
print(f"classes: {sorted(meta_df['finding'].unique())}")

unverified = meta_df['verified finding'].eq('No').sum()
verified = meta_df['verified finding'].eq('Yes').sum()
print(f"Not verified: {unverified}")
print(f"Verified: {verified}")

unverified_patient_ids = meta_df.loc[meta_df['verified finding'] == 'No', 'patient id'].unique()
assert len(unverified_patient_ids) == unverified # patient id is expected to be unique in metadata.csv
print(f"first unverified ID: {unverified_patient_ids[0]}")

verified_patient_ids = meta_df.loc[meta_df['verified finding'] == 'Yes', 'patient id'].unique()
assert len(verified_patient_ids) == verified # patient id is expected to be unique in metadata.csv
print(f"first verified ID: {verified_patient_ids[0]}")

imgs = [entry.name for entry in os.scandir(DATASET_DIR) if entry.is_file()]
print(f"total images: {len(imgs)}")
print(f"first 10 images: {imgs[:10]}")
imgs_of_1_patient = [img for img in imgs if verified_patient_ids[0] in img]
print(f"Images of patient ID {verified_patient_ids[0]}: {imgs_of_1_patient}")

## Investigating split dataset

In [ ]:
# Specify all the filepath of the dataset
DATA_DIR = "/kaggle/input/covidxct"
dirs = [entry.name for entry in os.scandir(DATA_DIR) if entry.is_dir()]
assert len(dirs) == 1 # expects to only have 1 folder that contains all the images

DATASET_DIR = path.join(DATA_DIR, dirs[0])
METADATA_CSV = path.join(DATA_DIR, "metadata.csv")

DATASET_NAME = "COVIDx_CT-3A"
TRAIN_SPLIT_FILE = path.join(DATA_DIR, f"train_{DATASET_NAME}.txt")
VAL_SPLIT_FILE = path.join(DATA_DIR, f"val_{DATASET_NAME}.txt")
TEST_SPLIT_FILE = path.join(DATA_DIR, f"test_{DATASET_NAME}.txt")
SPLIT_FILES = [TRAIN_SPLIT_FILE, VAL_SPLIT_FILE, TEST_SPLIT_FILE]

full_dataset = CovidNetDataset(DATASET_DIR, SPLIT_FILES)
full_data_len = len(full_dataset)
print(f"Length of full dataset: {full_data_len}")

SEED = 18
seed_everything(SEED)
BATCH_SIZE = 128

# # Defines ratios, w.r.t. whole dataset.
ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1
dummy_X = np.zeros(full_data_len)
indexes = np.arange(full_data_len)

# Produces test split. Uses train_test_split instead of StratifiedShuffleSplit to get x_remaining & y_remaining
# to be used in the next step. 
# Note that an additional indexes array is provided
x_remaining, X_test, y_remaining, Y_test, temp_train_index, test_index = train_test_split(
    dummy_X, full_dataset.targets, indexes, test_size=ratio_test, stratify=full_dataset.targets, random_state=SEED)
# train_index, test_index = next(
#     StratifiedShuffleSplit(n_splits=1, test_size=ratio_test, random_state=SEED).split(
#         dummy_X, full_dataset.targets
#     )
# )

print('*'*50)
print("temp_train")
print('*'*50)
print(f"First 10 index: {temp_train_index[:10]}")
print(f"First 10 label: {y_remaining[:10]}")
print(f"length of index: {len(temp_train_index)}")
print(">>>Distribution of labels:")
print(f"Normal: {y_remaining.count(0)}")
print(f"Pneunomia: {y_remaining.count(1)}")
print(f"Covid-19: {y_remaining.count(2)}")

print()
print('*'*50)
print("test")
print('*'*50)
print(f"First 10 index: {test_index[:10]}")
print(f"First 10 label: {Y_test[:10]}")
print(f"length of index: {len(test_index)}\n")
print(">>>Distribution of labels:")
print(f"Normal: {Y_test.count(0)}")
print(f"Pneunomia: {Y_test.count(1)}")
print(f"Covid-19: {Y_test.count(2)}")

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
X_train, X_val, Y_train, Y_val, train_index, val_index = train_test_split(
    x_remaining, y_remaining, temp_train_index, test_size=ratio_val_adjusted, stratify=y_remaining, random_state=SEED)

print()
print('*'*50)
print("train")
print('*'*50)
print(f"First 10 index: {train_index[:10]}")
print(f"First 10 label: {Y_train[:10]}")
print(f"length of index: {len(train_index)}\n")
print(">>>Distribution of labels:")
print(f"Normal: {Y_train.count(0)}")
print(f"Pneunomia: {Y_train.count(1)}")
print(f"Covid-19: {Y_train.count(2)}")

print()
print('*'*50)
print("val")
print('*'*50)
print(f"First 10 index: {val_index[:10]}")
print(f"First 10 label: {Y_val[:10]}")
print(f"length of index: {len(val_index)}\n")
print(">>>Distribution of labels:")
print(f"Normal: {Y_val.count(0)}")
print(f"Pneunomia: {Y_val.count(1)}")
print(f"Covid-19: {Y_val.count(2)}")

## Visualizing Dataloader

In [ ]:
import pprint
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array([0.485, 0.456, 0.406])
    # std = np.array([0.229, 0.224, 0.225])
    # inp = std * inp + mean
    # inp = np.clip(inp, 0, 1)
    plt.figure(figsize=[15, 15])
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

seed_everything(19)
class_names, _ = full_dataset.find_classes()
data, classes = next(iter(train_loader)) # note that it is normal for warning about clipping here if the image has been normalized
# out = torchvision.utils.make_grid(data)
# imshow(out)
# pp = pprint.PrettyPrinter(compact=True)
# pp.pprint([class_names[x] for x in classes])

print(f"Class: {classes[0]}")
out = data[0]
imshow(out)

## Trying Avalance

In [ ]:
# https://changhsinlee.com/colab-import-python/
!pip install requests
!pip install avalanche-lib

In [ ]:
import requests

# Save datagenerators as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/ContinualAI/avalanche/master/examples/pytorchcv_models.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('pytorchcv_models.py', 'w') as f:
    f.write(r.text)

# now we can import
import pytorchcv_models as pycv
from types import SimpleNamespace

args = SimpleNamespace()
args.cuda = 0
pycv.main(args)

## Pretrained Model

### Helper Functions

In [8]:
LOG_DIR = os.path.join(CURR_DIR, "log")
RESULT_DIR = os.path.join(CURR_DIR, 'result')
curr_model = ""

def log_to_file(txt=None, print_to_console_only=False):
  if txt is None:
    txt = ''
  txt += '\n'
  print(txt)
  if print_to_console_only:
    return
  if not path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
  full_path = os.path.join(LOG_DIR, f'{curr_model}.txt')
  with open(full_path, mode='a') as f:
    f.write(txt)
    
# https://discuss.pytorch.org/t/clearing-the-gpu-is-a-headache/84762
# Make sure to delete any references to tensor. Else this function will not have significant effect
def clean_vram():
    gc.collect()
    torch.cuda.empty_cache()

### Define Function to Initialize Deep Learning Models

In [9]:
model_constructors = {
  models.alexnet.__name__: models.alexnet, 
  models.squeezenet1_1.__name__: models.squeezenet1_1,
  models.resnet50.__name__: models.resnet50, 
  models.resnet101.__name__: models.resnet101,
  models.resnet152.__name__: models.resnet152, 
  models.resnext101_32x8d.__name__: models.resnext101_32x8d, 
  models.densenet201.__name__: models.densenet201, 
  models.googlenet.__name__: models.googlenet, 
  models.vgg16.__name__: models.vgg16, 
  models.vgg19.__name__: models.vgg19, 
  models.inception_v3.__name__: models.inception_v3, 
}

# Experiment around dropout & Learning Rate & different optimizer (Adam)
def init_model(name):
  if not path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)

  clean_vram()
  seed_everything()
  model = model_constructors[name](pretrained=True) 
  
  # fine-tune pretrain models to our usecase
  # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#initialize-and-reshape-the-networks
  NUM_CLASSES = len(class_names)
  DROPOUT = 0.5
  if name == models.alexnet.__name__ or name == models.vgg16.__name__ or name == models.vgg19.__name__:
    num_ftrs = model.classifier[6].in_features
    # model.classifier[6] = nn.Linear(num_ftrs, NUM_CLASSES)
    model.classifier[6] = nn.Sequential(
      nn.Dropout(DROPOUT),
      nn.Linear(num_ftrs, NUM_CLASSES)
    )
  elif name == models.densenet201.__name__:
    num_ftrs = model.classifier.in_features
    # model.classifier = nn.Linear(num_ftrs, NUM_CLASSES)
    model.classifier = nn.Sequential(
      nn.Dropout(DROPOUT),
      nn.Linear(num_ftrs, NUM_CLASSES)
    )
  elif name == models.squeezenet1_1.__name__:
    # model.classifier = nn.Conv2d(512, NUM_CLASSES, kernel_size=(1,1), stride=(1,1))
    model.classifier = nn.Sequential(
      nn.Dropout(DROPOUT),
      nn.Conv2d(512, NUM_CLASSES, kernel_size=(1,1), stride=(1,1))
    )
    model.num_classes = NUM_CLASSES
  elif name == models.inception_v3.__name__:
    auxLogits_num_ftrs = model.AuxLogits.fc.in_features
    # model.AuxLogits.fc = nn.Linear(auxLogits_num_ftrs, NUM_CLASSES)
    model.AuxLogits.fc = nn.Sequential(
      nn.Dropout(DROPOUT),
      nn.Linear(auxLogits_num_ftrs, NUM_CLASSES)
    )
    primary_num_ftrs = model.fc.in_features
    # model.fc = nn.Linear(primary_num_ftrs, NUM_CLASSES)
    model.fc = nn.Sequential(
      nn.Dropout(DROPOUT),
      nn.Linear(primary_num_ftrs, NUM_CLASSES)
    )
  else:
    # resnet, resnext & googlenet
    num_ftrs = model.fc.in_features
    # model.fc= nn.Linear(num_ftrs, NUM_CLASSES)
    model.fc = nn.Sequential(
      nn.Dropout(DROPOUT),
      nn.Linear(num_ftrs, NUM_CLASSES)
    )

  model = model.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer= optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
  if is_gpu_avail():
    # Use Automatic Mixed Precision as an attempt to solve CUDA out of memory and to speed things up
    # https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html#all-together-automatic-mixed-precision
    scaler = torch.cuda.amp.GradScaler()
  else:
    raise RuntimeError('This code only support machine with GPU.')

  # print('=====================================')
  print(f'{name} is initialized')
  # print('=====================================')
  # print(model)
  return model, criterion, optimizer, scaler

# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html
def save_model(perf_metrics, model, optimizer, scaler, history, model_path):
  torch.save({
    'perf_metrics': perf_metrics,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    "scaler_state_dict": scaler.state_dict(),
    'history': history,
    }, model_path)

def load_model(model, optimizer, scaler, model_path):
  checkpoint = torch.load(model_path)
  perf_metrics = checkpoint['perf_metrics']
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  scaler.load_state_dict(checkpoint['scaler_state_dict'])
  history = checkpoint['history']
  total_epoch = len(history) - 1

  return perf_metrics, model, optimizer, scaler, history, total_epoch

### Define Function to Train Models

In [10]:
# training and validation loops
def train(model,
    criterion,
    optimizer,
    scaler,
    train_dataloader,
    valid_dataloader,
    model_path,
    max_epochs_stop=10,
    n_epochs=400,
    min_epoch=300,
    print_every=1):
    
    epochs_no_improve = 0
    perf = {
        'best_epoch': 0,
        'valid_loss_min': np.Inf,
        'valid_best_acc': 0,
    }
    total_epoch = 0

    try:
        if os.path.exists(model_path):
            perf, model, optimizer, scaler, history, total_epoch = load_model(model, optimizer, scaler, model_path)
            log_to_file(f'Model has been trained for: {total_epoch} epochs.')
            log_to_file(f"Best epoch: {perf['best_epoch']} with loss: {perf['valid_loss_min']:.4f} and acc: {100 * perf['valid_best_acc']:.2f}%\n")
        else:
            history = []
            log_to_file(f'Starting Training from Scratch.\n')
    except:
        history = []
        log_to_file(f'exception: start from scratch.\n')

    overall_start = time.time()
    if total_epoch >= n_epochs:
        log_to_file(f'Model has been fully trained. n_epochs specified is: {n_epochs} epochs.')
        history = pd.DataFrame(
            history,
            columns=['train_loss', 'valid_loss','train_acc', 'valid_acc'])
        return model, history, perf

    seed_everything()

    # Main loop - continue training on where we left off if there's a saved model
    for epoch in range(total_epoch, n_epochs):
        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = time.time()
        for ii, (data, target) in enumerate (train_dataloader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()

            # only for inception_v3 - https://discuss.pytorch.org/t/why-auxiliary-logits-set-to-false-in-train-mode/40705/15
            with torch.cuda.amp.autocast():
              # output, aux_output = model(data)
              # loss1 = criterion(output, target)
              # loss2 = criterion(aux_output, target)
              # loss = loss1 + 0.4*loss2
              output = model(data)
              loss = criterion(output, target)
            # loss.backward()
            # optimizer.step()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * data.size(0)
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item() * data.size(0)
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_dataloader):.2f}% complete. {time.time() - start:.2f} seconds elapsed in epoch.', end="\r")
            
            # cleanup to save VRAM
            del data, target
#             clean_vram()

        # After training loops ends, start validation
        else:
            with torch.no_grad():
                model.eval()
                for data, target in valid_dataloader:
                    if is_gpu_avail():
                        data, target = data.cuda(), target.cuda()
                    output = model(data)
                    loss = criterion(output, target)
                    valid_loss += loss.item() * data.size(0)
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    valid_acc += accuracy.item() * data.size(0)
                    
                    # cleanup to save VRAM
                    del data, target
#                     clean_vram()
                train_loss = train_loss / train_data_size
                valid_loss = valid_loss / valid_data_size
                train_acc = train_acc / train_data_size
                valid_acc = valid_acc / valid_data_size
                history.append([train_loss, valid_loss,train_acc, valid_acc])
                if (epoch + 1) % print_every == 0:
                    log_to_file(f'Epoch: {epoch}', True)
                    log_to_file(
                        f'Training Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}',
                        True
                    )
                    log_to_file(
                        f'Training Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}% \n',
                        True
                    )
          
                if valid_loss < perf['valid_loss_min']:
                    epochs_no_improve = 0
                    perf['best_epoch'] = epoch
                    perf['valid_loss_min'] = valid_loss
                    perf['valid_best_acc'] = valid_acc
                    save_model(perf, model, optimizer, scaler, history, model_path)
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epoch > min_epoch and epochs_no_improve >= max_epochs_stop:
                        log_to_file(
                            f"\nEarly Stopping! Total epochs: {epoch}. Best epoch: {perf['best_epoch']} with loss: {perf['valid_loss_min']:.4f} and acc: {100 * perf['valid_best_acc']:.2f}%"
                        )
                        total_time = time.time() - overall_start
                        log_to_file(
                            f'{total_time:.4f} total seconds elapsed. {total_time / (epoch+1):.4f} seconds per epoch.'
                        )
                        log_to_file()

                        # Load the best state from saved model
                        _, model, optimizer, scaler, _, _ = load_model(model, optimizer, scaler, model_path)
                        # save the full history
                        save_model(perf, model, optimizer, scaler, history, model_path)

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history, perf
    
    total_time = time.time() - overall_start
    log_to_file(
        f"\nBest epoch: {perf['best_epoch']} with loss: {perf['valid_loss_min']:.4f} and acc: {100 * perf['valid_best_acc']:.4f}%"
    )
    log_to_file(
        f"{total_time:.4f} total seconds elapsed. {total_time / (perf['best_epoch']+1):.4f} seconds per epoch."
    )
    log_to_file()

    # Load the best state from saved model
    _, model, optimizer, scaler, _, _ = load_model(model, optimizer, scaler, model_path)
    # save the full history
    save_model(perf, model, optimizer, scaler, history, model_path)

    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss','train_acc', 'valid_acc'])
    
    return model, history, perf


def save_train_val_loss_graph(history, perf):
  plt.figure(figsize=(8, 6))
  for c in ['train_loss', 'valid_loss']:
      plt.plot(
          history[c], label=c)

  title = f'{curr_model} - Training and Validation Losses'
  full_path = os.path.join(RESULT_DIR, f'{title}.png')
  plt.xlabel('Epochs')
  plt.ylabel('Average Losses')
  plt.title(title)
  plt.axvline(x=perf['best_epoch'], color='r', label='best_epoch')
  plt.legend()
  plt.savefig(full_path, bbox_inches='tight')


def save_train_val_acc_graph(history, perf):
  plt.figure(figsize=(8, 6))
  for c in ['train_acc', 'valid_acc']:
      plt.plot(
          100 * history[c], label=c)
      
  title = f'{curr_model} - Training and Validation Accuracy'
  full_path = os.path.join(RESULT_DIR, f'{title}.png')
  plt.xlabel('Epochs')
  plt.ylabel('Average Accuracy')
  plt.title(title)
  plt.axvline(x=perf['best_epoch'], color='r', label='best_epoch')
  plt.legend()
  plt.savefig(full_path, bbox_inches='tight')

### Define Functions to Visualize Prediction

In [11]:
# confusion matrix 
def getConfusionMatrix(model, dataloader, is_test=False, show_image=False, print_to_console_only=False):
    model.eval()
    confusion_matrix=np.zeros((2,2),dtype=int)
    num_images=test_data_size
    
    with torch.no_grad():
        for i, (data,target) in enumerate(dataloader):
            data = data.to(device)
            target = target.to(device)
            
            output = model(data) 
            _, pred = torch.max(output, 1)
            
            for j in range(data.size()[0]): 
                if pred[j]==1 and target[j]==1:
                    term='TP'
                    confusion_matrix[0][0]+=1
                elif pred[j]==1 and target[j]==0:
                    term='FP'
                    confusion_matrix[1][0]+=1
                elif pred[j]==0 and target[j]==1:
                    term='FN'
                    confusion_matrix[0][1]+=1
                elif pred[j]==0 and target[j]==0:
                    term='TN'
                    confusion_matrix[1][1]+=1
            
                if show_image:
                    log_to_file(f'predicted: {class_names[pred[j]]}', print_to_console_only)
                    log_to_file(term, print_to_console_only)
                    imshow(data.cpu().data[j])
        
        log_to_file(None, print_to_console_only)
        category = 'Test' if is_test else 'Validation'
        log_to_file('=====================', print_to_console_only)
        log_to_file(f'{category} Results ', print_to_console_only)
        log_to_file('=====================', print_to_console_only)
        log_to_file('Confusion Matrix: ', print_to_console_only)
        log_to_file(np.array2string(confusion_matrix), print_to_console_only)
        log_to_file(None, print_to_console_only)

        log_to_file(f'Sensitivity: {100*confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[0][1])}', print_to_console_only)
        log_to_file(f'Specificity: {100*confusion_matrix[1][1]/(confusion_matrix[1][1]+confusion_matrix[1][0])}', print_to_console_only)
        log_to_file(f'PPV: {100*confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[1][0])}', print_to_console_only)
        log_to_file(f'NPV: {100*confusion_matrix[1][1]/(confusion_matrix[1][1]+confusion_matrix[0][1])}', print_to_console_only)
        log_to_file(f'Accuracy: {100*(confusion_matrix[0][0]+confusion_matrix[1][1])/(confusion_matrix[0][0]+confusion_matrix[0][1]+confusion_matrix[1][1]+confusion_matrix[1][0])}', print_to_console_only)
        log_to_file(f'F1-Score: {(2*confusion_matrix[0][0])/(2*confusion_matrix[0][0]+confusion_matrix[1][0]+confusion_matrix[0][1])}', print_to_console_only)
        log_to_file(None, print_to_console_only)
    return confusion_matrix


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_test_acc_n_loss_graph(model, dataloader, criterion):
  pass
  # NOT NEEDED YET
  # with torch.no_grad():
  #   model.eval()
  #   for data, target in dataloader:
  #       if is_gpu_avail():
  #           data, target = data.cuda(), target.cuda()
  #       output = model(data)
  #       loss = criterion(output, target)
  #       test_loss += loss.item() * data.size(0)
  #       _, pred = torch.max(output, dim=1)
  #       correct_tensor = pred.eq(target.data.view_as(pred))
  #       accuracy = torch.mean(
  #           correct_tensor.type(torch.FloatTensor))
  #       test_acc += accuracy.item() * data.size(0)
  #   train_loss = train_loss / train_data_size
  #   test_loss = test_loss / test_data_size
  #   train_acc = train_acc / train_data_size
  #   test_acc = test_acc / test_data_size


# def visualize_test_prediction(model):
#   covid_test_img_dir = '/content/drive/My Drive/data/test/covid/'
#   img_list = [Image.open(os.path.join(pth, f)).convert('RGB')
#       for pth, dirs, files in os.walk(covid_test_img_dir) for f in files]

#   # test_img_paths = ['/content/drive/My Drive/data/test/covid/2020.02.22.20024927-p19-68%3.png',
#   #                         '/content/drive/My Drive/data/test/covid/2020.02.22.20024927-p19-68%4.png',
#   #                         '/content/drive/My Drive/data/test/covid/2020.02.22.20024927-p19-68%5.png']
#   # img_list = [Image.open( img_path) for img_path in test_img_paths]

#   # log_to_file(img_list)

#   test_batch = torch.stack([image_transforms['test'](img).to(device)
#                               for img in img_list])
#   pred_logits_tensor = model(test_batch)
#   pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()

#   row = 12
#   col = 3
#   fig, axs = plt.subplots(row, col, figsize=(20, 50))
#   r = 0
#   c = 0
#   for i, img in enumerate(img_list):
#       if c >= col:
#         r += 1
#         c = 0
#       ax = axs[r, c]
#       ax.axis('off')
#       ax.set_title("{:.4f}% Covid, {:.4f}% NonCovid".format(100*pred_probs[i,0],
#                                                               100*pred_probs[i,1]))
#       ax.imshow(img)
#       c +=1

#   title = f'{curr_model} - Covid Image Prediction'
#   full_path = os.path.join(RESULT_DIR, f'{title}.png')
#   plt.savefig(full_path, bbox_inches='tight')


def getPredProbs(model, datasetStr, count, isSeeded=True):
  if isSeeded:
    seed_everything()
  
  dataset = data[datasetStr].samples
  img_list = []
  for i, (img_path, cls_idx) in enumerate(dataset):
    if i >= count:
      break
    img_list.append(Image.open(img_path).convert('RGB'))

  test_batch = torch.stack([image_transforms[datasetStr](img).to(device)
                              for img in img_list])
  pred_logits_tensor = model(test_batch)
  return F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()

### Run all models - Init Models + Training

In [ ]:
model_list = [
    models.alexnet.__name__, # 0
    models.squeezenet1_1.__name__, #1
    models.resnet50.__name__, # 2
    models.resnet101.__name__, # 3
    models.resnet152.__name__, # 4
    models.resnext101_32x8d.__name__, # 5
    models.densenet201.__name__, # 6
    models.googlenet.__name__, # 7
    models.vgg16.__name__, # 8
    models.vgg19.__name__, #9
    models.inception_v3.__name__, #10
]

for i in range(0,11):
  # https://github.com/pytorch/pytorch/issues/50198
  # skipped these because cannot use deterministic algorithm
#   skip_model = [0, 1, 5, 8, 9, 10]
  skip_model = [0, 1, 2, 3, 5, 6, 7, 8, 9, 10]
  if i in skip_model:
    continue
  curr_model = model_list[i]

  # Initialize model, criterion and optimizer
  model, criterion, optimizer, scaler = init_model(curr_model)

#   Training & Validation
  model, history, perf = train(
      model,
      criterion,
      optimizer,
      scaler,
      train_loader,
      val_loader,
      model_path=f'{path.join(RESULT_DIR, curr_model)}.pt',
      max_epochs_stop=5,  # Early stopping intialization
      n_epochs=1,
      min_epoch=1,
      print_every=10)

  save_train_val_loss_graph(history, perf)
  save_train_val_acc_graph(history, perf)
  getConfusionMatrix(model, val_loader)

env: PYTHONHASHSEED=18


Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

## Out of memory issue

- References
    - https://discuss.pytorch.org/t/using-main-ram-instead-of-vram/59344/3 
    - https://duckduckgo.com/?q=pytorch+colab+use+system+ram+instead+of+gpu+ram&ia=web
    - https://stackoverflow.com/questions/59129812/how-to-avoid-cuda-out-of-memory-in-pytorch
        - [CUDA Out of Memory discussion in kaggle forum](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/91081)
    - https://pytorch.org/docs/stable/notes/cuda.html#memory-management
    - [trick to debug tensor memory](https://forum.pyro.ai/t/a-trick-to-debug-tensor-memory/556)
- The fix
    - Delete unused tensor, force garbage collection and run `empty_cache()`
    - Set PYTORCH_CUDA_ALLOC_CONF to `max_split_size_mb:512`. This prevents the allocator to split block large than 512MB

In [ ]:
torch.cuda.memory_stats(device)
# print(torch.cuda.memory_summary(device))